In [122]:
from customizer import *
import keras
from keras import layers
import numpy as np
import random
import sys
import pandas as pd

In [132]:
#df=pd.read_csv('all_five_star_restaurants.csv')

In [133]:
#df.head()

,text,name
0,Amazing food!! I can't believe it almost taste...,South Florida Style Chicken & Ribs
1,"Great soul food period , it's get no better, c...",South Florida Style Chicken & Ribs
2,My favorite stop. It is what you would expect ...,Blimpie
3,This place is the best. They will make a sub t...,Blimpie
4,I really like their bread. Always soft inside...,Blimpie


In [125]:
#df=pd.read_csv('business.csv')
#df2=pd.read_csv('review.csv')
#five_star=df2[df2['stars']==5]
#restaurants = df[df['categories'].str.contains('Restaurants')]
#restaurants_clean=restaurants[['business_id','name']]
#all_five_star_restaurants=pd.merge(restaurants_clean, five_star, on='business_id')
#all_five_star_restaurants=all_five_star_restaurants[['text', 'name']]

In [131]:
#all_five_star_restaurants.to_csv('all_five_star_restaurants.csv', index=False)

In [2]:
text = open('short_reviews_shuffle.txt').read()

In [8]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def findStrAfterStr(myString, searchText, afterText):
    after_index = myString.index(afterText)
    return myString.find(searchText, after_index)

In [ ]:
def get_second_index(input_string, sub_string):
    return input_string.index(sub_string, input_string.index(sub_string) + 1)

In [4]:
#chars = sorted(list(set(text)))
# Dictionary mapping unique characters to their index in `chars`
chars=['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~']

In [5]:
char_indices = dict((char, chars.index(char)) for char in chars)
maxlen=60
step=1

In [6]:
model = keras.models.Sequential()
model.add(layers.LSTM(1024, input_shape=(maxlen, len(chars)),return_sequences=True))
model.add(layers.LSTM(1024, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))
model.load_weights("Sep-26-all-00-0.7359.hdf5")

optimizer = keras.optimizers.Adam(lr=0.0002)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [113]:
print('Coming up with two ideas for you...')
start_index = random.randint(0, len(text) - maxlen - 1)
generated_text = text[start_index: start_index + maxlen]

final=generated_text+''

#for temperature in [0.5, 0.8, 1.0]:
for temperature in [0.8]:
    #print('------ temperature:', temperature)
    #sys.stdout.write(generated_text)

        # We generate 600 characters
    for i in range(600):
        sampled = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.

        preds = model.predict(sampled, verbose=0)[0]
        next_index = sample(preds, temperature)
        next_char = chars[next_index]
        final+=next_char
        
        
        generated_text += next_char
        generated_text = generated_text[1:]

        #sys.stdout.write(next_char)
        #sys.stdout.flush()
temp=personalized_clean_up(final, 'cold-brew,espresso,coffee')
start=temp.find('SOR')
stop=findStrAfterStr(temp, 'EOR', 'SOR')
end_first=temp[start+4:stop]

new=temp[get_second_index(temp, 'SOR')+4:]
ending=new.find('EOR')
print(temp[start+4:stop])
print("")
print(new[:ending])

Coming up with two ideas for you...


/Users/andrewmanser/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


It's my favourite coffee, I always get the greek coffee. It is a sweeter than coffee and it stays 400 minutes of ordering. I also had the Tangerine Rolled Spicy Lean Roll and it was amazing. espresso service... Will definitely be back... 

coffee food, ambiance, and service! We had a coupon from Viagias, great food and service great atmosphere and the service is very good for the price. I was wrong. It was perfect. 


In [114]:
print(final)

 wonderful.<EOR>"
"<SOR>It's my favourite location, I always get the greek salad. It is a sweeter than fettuccine and it stays 400 minutes of ordering. I also had the Tangerine Rolled Spicy Lean Roll and it was amazing. Great service...  Will definitely be back...<EOR>"
"<SOR>Great food, ambiance, and service! We had a coupon from Viagias, great food and service great atmosphere and the service is very good for the price.  I was wrong.  It was perfect.<EOR>"
"<SOR>My wife and I come here all the time. Love the people there and the food.<EOR>"
"<SOR>Love this place!!! Good for breakfast. Very clean, staff is very friendly.<EOR>"
"<SOR>I'm one of the way


In [115]:
#PRINT RANDOM REVIEWS
# Select a text seed at random
def random_reviews():
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Coming up with several reviews for you...')

    for temperature in [0.8]:
        sys.stdout.write(generated_text)

        # We generate 600 characters
        for i in range(600):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [117]:
random_reviews()

Coming up with several reviews for you...
zona.<EOR>"
"<SOR>Great food and service was awesome as well as ambian

/Users/andrewmanser/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


ce as well.  Loved it<EOR>"
"<SOR>Great BBQ.  Great salads and fries.  Food was amazing and Spanish Buffalo mushroom Burger was absolutely delicious! Everything came out within 5 minutes, but that's our server and she was really sweet, and the food was amazing!!! The portions were big and packed during a day of when I was still thinking about it. The tali and potato wedges are also amazing. We will miss this time or hold more to this gem.<EOR>"
"<SOR>Remember to get the punch of light lunch special you can not go wrong. This place is the real deal.<EOR>"
"<SOR>This place is awesome. 


In [120]:
#PRINT USER CUSTOMIZED REVIEWS
def user_custom(foods):
    #foods is a string separated by commas
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Coming up with two ideas for you...')
    
    final=generated_text+''
    
    for temperature in [0.8]:
        

        # We generate 600 characters
        for i in range(600):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            final+=next_char

            generated_text += next_char
            generated_text = generated_text[1:]
    #print first review, then second via SOR/EOR 
    temp=personalized_clean_up(final, foods)
    start=temp.find('SOR')
    stop=findStrAfterStr(temp, 'EOR', 'SOR')
    end_first=temp[start+4:stop]

    new=temp[get_second_index(temp, 'SOR')+4:]
    ending=new.find('EOR')
    print(temp[start+4:stop])
    print("")
    print(new[:ending])

In [121]:
user_custom('candy,soda,sugar,gummies')

Coming up with two ideas for you...


/Users/andrewmanser/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


Just stopped by for a quick sugar. The portions were great. Highly recommend!! 

Love the soda and sugar. The candy is very fresh and the staff is very friendly. It looks sketchy sugar as the food is amazing and the service is great as well. They also have some interesting candy specials. 


In [135]:
#PRINT database CUSTOMIZED REVIEWS
def df_custom(restaurant):
    #restaurant is a string exactly the same as in the dataframe
    df=pd.read_csv('all_five_star_restaurants.csv')
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('Coming up with two ideas for you...')
    
    final=generated_text+''
    
    for temperature in [0.8]:
        

        # We generate 600 characters
        for i in range(600):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]
            final+=next_char

            generated_text += next_char
            generated_text = generated_text[1:]
    #print first review, then second via SOR/EOR
    items=pull_restaurant_review(restaurant, df)  
    
    temp=clean_up(final, items)
    start=temp.find('SOR')
    stop=findStrAfterStr(temp, 'EOR', 'SOR')
    end_first=temp[start+4:stop]

    new=temp[get_second_index(temp, 'SOR')+4:]
    ending=new.find('EOR')
    print(temp[start+4:stop])
    print("")
    print(new[:ending])

In [136]:
df_custom('Subway')

Coming up with two ideas for you...


/Users/andrewmanser/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()


Ben location, and the best prepared around south!!! You won't regret the true little area that shows the dancers on the place burritos know the food come out great!! 

Best vegan food I've had since visiting the sub satellite. You won't be able to stop by and fall in love their menu. 
